<center>
    <h1 style="font-size: 12; font-weight: 900; ">--- 5 ---<h1>
    <h2>CSiAPI - ETABS y SAP2000</h2>
    <!-- <p><strong></strong</p> -->
</center>

In [7]:
import os, sys

import comtypes.client
import ctypes

7. Funciones utiles de limpiesa y restamblecimiento

In [8]:
# Elimina objetos existentes FrameObj
def delete_all_frames(smodel):
    # Get the names of all frame objects
    ret, all_frames, ret = smodel.FrameObj.GetNameList()

    # Delete each frame object
    for frame in all_frames:
        smodel.FrameObj.Delete(frame)

    print("All frames deleted successfully")

In [9]:
# Generacion e la geometria de la estructura
def create_structural_system(smodel, bay_width=10, story_height=10, num_bays=2, num_stories=2):
    # Delete existing frames
    delete_all_frames(smodel)
    base_points = []

    # Define material and section properties
    smodel.PropMaterial.SetMaterial("Concrete", 2)
    smodel.PropFrame.SetRectangle("Beam", "Concrete", 0.3, 0.5)
    smodel.PropFrame.SetRectangle("Column", "Concrete", 0.5, 0.5)

    # Create columns and beams
    for i in range(num_bays + 1):
        for j in range(num_stories):
            x_coord = i * bay_width
            z_coord = j * story_height
            smodel.FrameObj.AddByCoord(x_coord, 0, z_coord, x_coord, 0, z_coord + story_height, "", "Column", f"C{i}{j}")
            if j == 0:
                base_points.append(f"{x_coord}_0_{z_coord}")

    # Create beams
    for j in range(1, num_stories + 1):
        z_coord = j * story_height
        for i in range(num_bays):
            x_start = i * bay_width
            x_end = (i + 1) * bay_width
            smodel.FrameObj.AddByCoord(x_start, 0, z_coord, x_end, 0, z_coord, "", "Beam", f"B{j}{i}")

    print("Structural system created successfully")
    return base_points

8. Analisis y solicitud de resultados

In [10]:
def run_analysis_and_get_joint_reactions(smodel):
    smodel.Analyze.RunAnalysis()
    smodel.Results.Setup.DeselectAllCasesAndCombosForOutput()
    smodel.Results.Setup.SetCaseSelectedForOutput("Dead")
    smodel.Results.Setup.SetCaseSelectedForOutput("Live")

    for each in base_points:
        print(smodel.Results.JointReact(each, 0))

Desbloque del modelo para implementar modificaciones

In [11]:
def unlocked_model(smodel):
    # Unlock the model before running the analysis
    smodel.SetModelIsLocked(False)

In [ ]:
# Initialize SAP2000
helper = comtypes.client.CreateObject('SAP2000v1.Helper')
helper = helper.QueryInterface(comtypes.gen.SAP2000v1.cHelper)
Sap2000Object = helper.CreateObjectProgID("CSI.SAP2000.API.SapObject")
Sap2000Object.ApplicationStart()
smodel = Sap2000Object.SapModel

# Initialize the model
smodel.InitializeNewModel()
KN_m_C = 6
smodel.SetPresentUnits(KN_m_C)
smodel.File.NewBlank()

# Save the model before analysis
# smodel.File.Save(r"C:\path\to\save\example.sdb")
# Se definen registros de salida
projectPath = r'C:\\CSi_API_Example';
if not os.path.exists(projectPath):
    try: os.makedirs(projectPath)
    except OSError: pass
ModelPath = projectPath + os.sep + 'Frame-e1.edb'
# save model | guardar nuestro modelo
ret = smodel.File.Save(ModelPath)

# Create the structural system and apply loads
base_points = create_structural_system(
    smodel, 
    bay_width=4, 
    story_height=8, 
    num_bays=9, 
    num_stories=18
    )

# Run the analysis and get joint reactions
run_analysis_and_get_joint_reactions(smodel)

# Unlock and run the analysis again (if needed)
unlocked_model(smodel)
run_analysis_and_get_joint_reactions(smodel)

All frames deleted successfully
Structural system created successfully
[0, (), (), (), (), (), (), (), (), (), (), (), 1]
[0, (), (), (), (), (), (), (), (), (), (), (), 1]
[0, (), (), (), (), (), (), (), (), (), (), (), 1]
[0, (), (), (), (), (), (), (), (), (), (), (), 1]
[0, (), (), (), (), (), (), (), (), (), (), (), 1]
[0, (), (), (), (), (), (), (), (), (), (), (), 1]
[0, (), (), (), (), (), (), (), (), (), (), (), 1]
[0, (), (), (), (), (), (), (), (), (), (), (), 1]
[0, (), (), (), (), (), (), (), (), (), (), (), 1]
[0, (), (), (), (), (), (), (), (), (), (), (), 1]
[0, (), (), (), (), (), (), (), (), (), (), (), 1]
[0, (), (), (), (), (), (), (), (), (), (), (), 1]
[0, (), (), (), (), (), (), (), (), (), (), (), 1]
[0, (), (), (), (), (), (), (), (), (), (), (), 1]
[0, (), (), (), (), (), (), (), (), (), (), (), 1]
[0, (), (), (), (), (), (), (), (), (), (), (), 1]
[0, (), (), (), (), (), (), (), (), (), (), (), 1]
[0, (), (), (), (), (), (), (), (), (), (), (), 1]
[0, (), (),

In [ ]:
if (Sap2000Object.ApplicationExit(False)):
    print("SAP2000, No se pudo cerrar")
else:
    print("Se cerro con exito.")

smodel = None
del smodel

Se cerro con exito.
